In [1]:
%load_ext dotenv
%dotenv

import os
from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether

USE_CHINESE=True

In [2]:
if not USE_CHINESE:
    # Use Mixtral
    model = ChatOpenAI(
        base_url="https://api.together.xyz/v1",
        api_key=os.environ["KEY_TOGETHERAI"],
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",)
else:
    # Use Qwen
    model = ChatTogether(
        together_api_key=os.environ['KEY_TOGETHERAI'],
        model="Qwen/Qwen1.5-72B-Chat",
    )

In [3]:
%reload_ext dotenv

In [4]:
# https://python.langchain.com/v0.2/docs/tutorials/rag/
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
loader = PyMuPDFLoader("concisehistory-hk.pdf")
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
print(docs[10])

In [6]:
hfe = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh-v1.5")
"""
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
"""
vectorstore = Chroma.from_documents(documents=docs, embedding=hfe)

/Users/sim/venv/langchain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/sim/venv/langchain/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_kwargs={'k': 6})
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
rag_chain.invoke("請介紹一下香港島的歷史，請以繁體中文作答")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'香港島的歷史可追溯至1841年，當時英國海軍登陸該島並佔領，根據1842年的《南京條約》被永久割讓給英國。在1997年前，香港一直受英國管治，除了在第二次世界大戰中被日本短期佔領。同年7月1日，香港成為中國的特別行政區。香港島早期的居民主要是農民、漁民和海盜，隨著時間發展，該地成為繁華的港口和大都市。在殖民地時期前，香港已有較多居民並有較為發達的農村和墟市。'

In [13]:
rag_chain.invoke("請簡單介紹一下張保仔在香港歷史中的角色和影響，必須要以繁體中文作答")

'張保仔是香港歷史中的一位著名海盜，他在19世紀初期以香港島為基地，利用山峰進行覘探和劫掠过往船舶，對当时的海上貿易造成一定影響。他的活動顯示了香港在清朝時與海盜活動的关联，而後随着英國的抵達及殖民地的建立，香港的治安才得以改善。'

In [14]:
rag_chain.invoke("請詳單講述一下彭定康在香港歷史中的角色和影響，必須要以繁體中文作答")

'彭定康是英國最後一位香港總督，他是首位政治任命的港督，因在1992年提出政治改革而知名。他的改革包括擴大立法局直選議席、取消委任議席、改選區制度及降低投票年齡，目的是提升香港的民主代表性並安定民心。這些改革虽受到香港自由派的歡迎，但超越了中國的接受範圍，被視為違反《聯合聲明》。彭定康的行動加劇了中英之間的緊張，中國在他離任後廢除了由他的改革選出的立法局，並設立臨時立法會。他的角色和影響是複雜的，既被批評破壞了中英關係，也被人視為推動香港民主的關鍵人物。'